# Implementation and Analysis of KNN with a Dataset







## 1) Dataset

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split


data = pd.read_csv("cybersecurity_attacks.csv")
data_proc = data
data

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,...,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source
0,2023-05-30 06:33:58,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis ius...,...,Logged,Low,Reyansh Dugal,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment A,"Jamshedpur, Sikkim",150.9.97.135,Log Data,NaN,Server
1,2020-08-26 07:08:30,78.199.217.198,66.191.137.154,17245,48166,ICMP,1174,Data,HTTP,Aperiam quos modi officiis veritatis rem. Omni...,...,Blocked,Low,Sumer Rana,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment B,"Bilaspur, Nagaland",NaN,Log Data,NaN,Firewall
2,2022-11-13 08:23:25,63.79.210.48,198.219.82.17,16811,53600,UDP,306,Control,HTTP,Perferendis sapiente vitae soluta. Hic delectu...,...,Ignored,Low,Himmat Karpe,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Segment C,"Bokaro, Rajasthan",114.133.48.179,Log Data,Alert Data,Firewall
3,2023-07-02 10:38:46,163.42.196.10,101.228.192.255,20018,32534,UDP,385,Data,HTTP,Totam maxime beatae expedita explicabo porro l...,...,Blocked,Medium,Fateh Kibe,Mozilla/5.0 (Macintosh; PPC Mac OS X 10_11_5; ...,Segment B,"Jaunpur, Rajasthan",NaN,NaN,Alert Data,Firewall
4,2023-07-16 13:11:07,71.166.185.76,189.243.174.238,6131,26646,TCP,1462,Data,DNS,Odit nesciunt dolorem nisi iste iusto. Animi v...,...,Blocked,Low,Dhanush Chad,Mozilla/5.0 (compatible; MSIE 5.0; Windows NT ...,Segment C,"Anantapur, Tripura",149.6.110.119,NaN,Alert Data,Firewall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,2023-05-26 14:08:42,26.36.109.26,121.100.75.240,31005,6764,UDP,1428,Control,HTTP,Quibusdam ullam consequatur consequuntur accus...,...,Logged,Medium,Adira Madan,Mozilla/5.0 (iPad; CPU iPad OS 14_2_1 like Mac...,Segment A,"Nashik, Manipur",NaN,Log Data,Alert Data,Firewall
39996,2023-03-27 00:38:27,17.21.163.81,196.108.134.78,2553,28091,UDP,1184,Control,HTTP,Quaerat neque esse. Animi expedita natus commo...,...,Logged,High,Rati Dara,Mozilla/5.0 (Windows; U; Windows 98; Win 9x 4....,Segment C,"Vadodara, Mizoram",60.51.30.46,Log Data,NaN,Firewall
39997,2022-03-31 01:45:49,162.35.217.57,98.107.0.15,22505,25152,UDP,1043,Data,DNS,Enim at aspernatur illum. Saepe numquam eligen...,...,Blocked,Low,Samiha Joshi,Mozilla/5.0 (Windows; U; Windows NT 4.0) Apple...,Segment C,"Mahbubnagar, Himachal Pradesh",NaN,Log Data,Alert Data,Server
39998,2023-09-22 18:32:38,208.72.233.205,173.79.112.252,20013,2703,UDP,483,Data,FTP,Officiis dolorem sed harum provident earum dis...,...,Ignored,Low,Rasha Chauhan,Mozilla/5.0 (X11; Linux i686) AppleWebKit/536....,Segment B,"Rourkela, Arunachal Pradesh",137.76.130.8,Log Data,NaN,Server


In [2]:
data_proc = data_proc.drop(columns=["Payload Data","Timestamp","Source IP Address","Destination IP Address","Source Port","Destination Port","Device Information","Proxy Information","User Information","Geo-location Data"])
data_proc["Malware Indicators"].fillna("Not Detected", inplace=True)
data_proc["Alerts/Warnings"].fillna("Not Detected", inplace=True)
data_proc["Firewall Logs"].fillna("Not Detected", inplace=True)
data_proc["IDS/IPS Alerts"].fillna("Not Detected", inplace=True)

data_proc

,Protocol,Packet Length,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,Severity Level,Network Segment,Firewall Logs,IDS/IPS Alerts,Log Source
0,ICMP,503,Data,HTTP,IoC Detected,28.67,Not Detected,Malware,Known Pattern B,Logged,Low,Segment A,Log Data,Not Detected,Server
1,ICMP,1174,Data,HTTP,IoC Detected,51.50,Not Detected,Malware,Known Pattern A,Blocked,Low,Segment B,Log Data,Not Detected,Firewall
2,UDP,306,Control,HTTP,IoC Detected,87.42,Alert Triggered,DDoS,Known Pattern B,Ignored,Low,Segment C,Log Data,Alert Data,Firewall
3,UDP,385,Data,HTTP,Not Detected,15.79,Alert Triggered,Malware,Known Pattern B,Blocked,Medium,Segment B,Not Detected,Alert Data,Firewall
4,TCP,1462,Data,DNS,Not Detected,0.52,Alert Triggered,DDoS,Known Pattern B,Blocked,Low,Segment C,Not Detected,Alert Data,Firewall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,UDP,1428,Control,HTTP,IoC Detected,39.28,Not Detected,DDoS,Known Pattern A,Logged,Medium,Segment A,Log Data,Alert Data,Firewall
39996,UDP,1184,Control,HTTP,IoC Detected,27.25,Not Detected,DDoS,Known Pattern A,Logged,High,Segment C,Log Data,Not Detected,Firewall
39997,UDP,1043,Data,DNS,IoC Detected,31.01,Not Detected,DDoS,Known Pattern B,Blocked,Low,Segment C,Log Data,Alert Data,Server
39998,UDP,483,Data,FTP,IoC Detected,97.85,Alert Triggered,Malware,Known Pattern B,Ignored,Low,Segment B,Log Data,Not Detected,Server


In [3]:
# drop label columns
X = data_proc.drop(columns=["Attack Type"])

# isolate y
y = data_proc["Attack Type"]

# split in Train-set(80%) and Testing-set(20%)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42) 

In [4]:
X_train.info(), X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26800 entries, 8451 to 15795
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Protocol            26800 non-null  object 
 1   Packet Length       26800 non-null  int64  
 2   Packet Type         26800 non-null  object 
 3   Traffic Type        26800 non-null  object 
 4   Malware Indicators  26800 non-null  object 
 5   Anomaly Scores      26800 non-null  float64
 6   Alerts/Warnings     26800 non-null  object 
 7   Attack Signature    26800 non-null  object 
 8   Action Taken        26800 non-null  object 
 9   Severity Level      26800 non-null  object 
 10  Network Segment     26800 non-null  object 
 11  Firewall Logs       26800 non-null  object 
 12  IDS/IPS Alerts      26800 non-null  object 
 13  Log Source          26800 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 3.1+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 1

(None, None)

## 2) Processing the Dataset 
In this step, we can begin creating two functions:

1. **process_data_x** : This function analyzes and cleans the dataset data, starting with the training set and ending with the testing set. The analysis is divided to preserve the accurate analysis of the data.
2. **process_data_y** : In this function, we will create and categorize data classes, placing them all into an array.


### 2.1) Process X

In [5]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

def process_data_x(train, test):
    # Numerical feature in my Dataset
    numerical_idx = ["Packet Length", "Anomaly Scores"]
    
    # convert numeric integer to float and concat them with already float feature 
     # There are no NaN element in these feature
    X_train = pd.concat([pd.to_numeric(train[numerical_idx[0]],downcast='float'), train[numerical_idx[1]]], axis=1)
    
    oneHot_idx = train.columns.drop(['Packet Length', 'Anomaly Scores','Severity Level'])
    
    oh = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    ord = OrdinalEncoder(categories=[['Low','Medium','High']])
    
    ord_encoded = ord.fit_transform(train[['Severity Level']])
    oneHot_encoded = oh.fit_transform(train[oneHot_idx])
    
    for i, col in enumerate(oh.get_feature_names_out()):
        X_train[col] = oneHot_encoded[:,i]
   
    for i, col in enumerate(ord.get_feature_names_out()):
        X_train[col] = ord_encoded[:,i]
        
    # --------------
    # process test
    X_test = pd.concat([pd.to_numeric(test[numerical_idx[0]],downcast='float'), test[numerical_idx[1]]], axis=1)
    
    oneHot_idx = test.columns.drop(['Packet Length', 'Anomaly Scores','Severity Level'])
    
    oh = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    ord = OrdinalEncoder(categories=[['Low','Medium','High']])
    
    ord_encoded = ord.fit_transform(test[['Severity Level']])
    oneHot_encoded = oh.fit_transform(test[oneHot_idx])
    
    for i, col in enumerate(oh.get_feature_names_out()):
        X_test[col] = oneHot_encoded[:,i]
   
    for i, col in enumerate(ord.get_feature_names_out()):
        X_test[col] = ord_encoded[:,i]
    
    
    return X_train, X_test

In [6]:
X_train_enc, X_test_enc = process_data_x(X_train, X_test)

In [7]:
X_test_enc

,Packet Length,Anomaly Scores,Protocol_ICMP,Protocol_TCP,Protocol_UDP,Packet Type_Control,Packet Type_Data,Traffic Type_DNS,Traffic Type_FTP,Traffic Type_HTTP,...,Network Segment_Segment A,Network Segment_Segment B,Network Segment_Segment C,Firewall Logs_Log Data,Firewall Logs_Not Detected,IDS/IPS Alerts_Alert Data,IDS/IPS Alerts_Not Detected,Log Source_Firewall,Log Source_Server,Severity Level
32823,1402.0,69.21,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
16298,855.0,55.69,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
28505,297.0,32.58,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0
6689,1090.0,9.42,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0
26893,330.0,13.88,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35934,811.0,75.89,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0
403,658.0,79.29,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
35953,1002.0,51.24,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
14218,423.0,94.74,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0


In [8]:
X_train_enc

,Packet Length,Anomaly Scores,Protocol_ICMP,Protocol_TCP,Protocol_UDP,Packet Type_Control,Packet Type_Data,Traffic Type_DNS,Traffic Type_FTP,Traffic Type_HTTP,...,Network Segment_Segment A,Network Segment_Segment B,Network Segment_Segment C,Firewall Logs_Log Data,Firewall Logs_Not Detected,IDS/IPS Alerts_Alert Data,IDS/IPS Alerts_Not Detected,Log Source_Firewall,Log Source_Server,Severity Level
8451,271.0,4.99,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
28293,543.0,16.16,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0
9948,775.0,18.83,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0
32270,744.0,17.36,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
39564,708.0,31.74,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1214.0,69.40,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0
11284,1246.0,80.79,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0
38158,369.0,82.18,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
860,168.0,17.07,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0


### 2.2) Process Y

In [9]:
label_enc = LabelEncoder()

In [10]:
def process_data_y(train,test):
    
    y_train_enc = label_enc.fit_transform(train)
    y_test_enc = label_enc.transform(test)
    
    return  y_train_enc,y_test_enc

In [11]:
y_train_enc, y_test_enc = process_data_y(y_train,y_test)

In [12]:
y_test_enc, y_train_enc, label_enc.classes_

(array([2, 0, 0, ..., 1, 1, 0]),
 array([0, 1, 2, ..., 2, 1, 2]),
 array(['DDoS', 'Intrusion', 'Malware'], dtype=object))

## 3) Valuation and Training

Now I can begin the training process. After separating the dataset into training and testing sets, I can start to evaluate and train it.

### 3.1) Accuracy and K-NN
First of all, we need to determine the accuracy baseline, as we will aim to enhance it with our model.

In [13]:
%matplotlib inline

import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

kNN = KNeighborsClassifier(n_neighbors=10)

kNN.fit(X_train_enc,y_train_enc)

y_pred = kNN.predict(X_test_enc)

acc = accuracy_score(y_true=y_test_enc, y_pred=y_pred)
print (f"Accuracy {acc:.3f}")


Accuracy 0.333


Now lest begin to create the accuracy function, that allows us to see the accuracy according to the K-NN Model

In [14]:
X = X_train_enc.loc[:,'Packet Length' :]
for f in X.columns:
    print ("Feature {:2s} -> Scale: {:7.2f}".format(f, X.loc[:,f].max() - X.loc[:,f].min()) )

Feature Packet Length -> Scale: 1436.00
Feature Anomaly Scores -> Scale:  100.00
Feature Protocol_ICMP -> Scale:    1.00
Feature Protocol_TCP -> Scale:    1.00
Feature Protocol_UDP -> Scale:    1.00
Feature Packet Type_Control -> Scale:    1.00
Feature Packet Type_Data -> Scale:    1.00
Feature Traffic Type_DNS -> Scale:    1.00
Feature Traffic Type_FTP -> Scale:    1.00
Feature Traffic Type_HTTP -> Scale:    1.00
Feature Malware Indicators_IoC Detected -> Scale:    1.00
Feature Malware Indicators_Not Detected -> Scale:    1.00
Feature Alerts/Warnings_Alert Triggered -> Scale:    1.00
Feature Alerts/Warnings_Not Detected -> Scale:    1.00
Feature Attack Signature_Known Pattern A -> Scale:    1.00
Feature Attack Signature_Known Pattern B -> Scale:    1.00
Feature Action Taken_Blocked -> Scale:    1.00
Feature Action Taken_Ignored -> Scale:    1.00
Feature Action Taken_Logged -> Scale:    1.00
Feature Network Segment_Segment A -> Scale:    1.00
Feature Network Segment_Segment B -> Scale:

In [15]:
X_train_enc

,Packet Length,Anomaly Scores,Protocol_ICMP,Protocol_TCP,Protocol_UDP,Packet Type_Control,Packet Type_Data,Traffic Type_DNS,Traffic Type_FTP,Traffic Type_HTTP,...,Network Segment_Segment A,Network Segment_Segment B,Network Segment_Segment C,Firewall Logs_Log Data,Firewall Logs_Not Detected,IDS/IPS Alerts_Alert Data,IDS/IPS Alerts_Not Detected,Log Source_Firewall,Log Source_Server,Severity Level
8451,271.0,4.99,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
28293,543.0,16.16,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0
9948,775.0,18.83,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0
32270,744.0,17.36,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
39564,708.0,31.74,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1214.0,69.40,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0
11284,1246.0,80.79,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0
38158,369.0,82.18,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
860,168.0,17.07,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0
